In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [5]:
# Image dimensions
IMG_SIZE = (48, 48)
BATCH_SIZE = 32

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load train and test datasets
train_generator = train_datagen.flow_from_directory(
    "dataset/fer/train", 
    target_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    color_mode="rgb", 
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    "dataset/fer/test", 
    target_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    color_mode="rgb", 
    class_mode="categorical"
)

num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Number of classes: 7


In [6]:
# Load VGG19 (pretrained on ImageNet)
base_model = VGG19(weights="imagenet", include_top=False, input_shape=(48, 48, 3))

# Freeze base model layers
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")  # Output layer
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

# Model summary
model.summary()

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)              │ (None, 1, 1, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,157,511 (76.89 MB)

 Trainable params: 133,127 (520.03 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [7]:
EPOCHS = 15

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS
)

Epoch 1/15


/Users/ayushidubey/Desktop/College/Semester-6/NNDL/labs/project/venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


898/898 ━━━━━━━━━━━━━━━━━━━━ 186s 207ms/step - accuracy: 0.2327 - loss: 1.9227 - val_accuracy: 0.2930 - val_loss: 1.7164
Epoch 2/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 187s 209ms/step - accuracy: 0.2758 - loss: 1.7603 - val_accuracy: 0.2974 - val_loss: 1.7112
Epoch 3/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 196s 218ms/step - accuracy: 0.2843 - loss: 1.7463 - val_accuracy: 0.3029 - val_loss: 1.7031
Epoch 4/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 202s 225ms/step - accuracy: 0.2887 - loss: 1.7478 - val_accuracy: 0.3111 - val_loss: 1.6827
Epoch 5/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 196s 218ms/step - accuracy: 0.2940 - loss: 1.7393 - val_accuracy: 0.3091 - val_loss: 1.6727
Epoch 6/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 186s 207ms/step - accuracy: 0.2928 - loss: 1.7415 - val_accuracy: 0.3155 - val_loss: 1.6755
Epoch 7/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 201s 224ms/step - accuracy: 0.2949 - loss: 1.7378 - val_accuracy: 0.3168 - val_loss: 1.6669
Epoch 8/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 206s 229ms/step - accuracy: 0.2994 - loss: 1.72

In [8]:
# Get train and test accuracies
train_loss, train_acc = model.evaluate(train_generator)
test_loss, test_acc = model.evaluate(test_generator)

print(f"Train Accuracy: {train_acc * 100:.2f}%")
print(f"Test Accuracy: {test_acc * 100:.2f}%")


898/898 ━━━━━━━━━━━━━━━━━━━━ 149s 166ms/step - accuracy: 0.2979 - loss: 1.7086
225/225 ━━━━━━━━━━━━━━━━━━━━ 37s 164ms/step - accuracy: 0.3088 - loss: 1.6665
Train Accuracy: 29.79%
Test Accuracy: 30.44%


In [9]:
# Assuming you have the true labels and predicted labels for both train and test datasets
train_predictions = model.predict(train_generator)
test_predictions = model.predict(test_generator)

# Convert predictions to class labels (if necessary, depends on your model output)
train_predictions_labels = np.argmax(train_predictions, axis=1)
test_predictions_labels = np.argmax(test_predictions, axis=1)

# Assuming your labels are categorical and represented as integers
print("Train Classification Report:")
print(classification_report(train_generator.classes, train_predictions_labels))

print("Test Classification Report:")
print(classification_report(test_generator.classes, test_predictions_labels))

# You can also extract individual metrics:
# For train
train_report = classification_report(train_generator.classes, train_predictions_labels, output_dict=True)
print(f"Train Precision: {train_report['weighted avg']['precision']:.2f}")
print(f"Train Recall: {train_report['weighted avg']['recall']:.2f}")
print(f"Train F1-Score: {train_report['weighted avg']['f1-score']:.2f}")

# For test
test_report = classification_report(test_generator.classes, test_predictions_labels, output_dict=True)
print(f"Test Precision: {test_report['weighted avg']['precision']:.2f}")
print(f"Test Recall: {test_report['weighted avg']['recall']:.2f}")
print(f"Test F1-Score: {test_report['weighted avg']['f1-score']:.2f}")


898/898 ━━━━━━━━━━━━━━━━━━━━ 158s 176ms/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 37s 166ms/step
Train Classification Report:
              precision    recall  f1-score   support

           0       0.26      0.00      0.01      3995
           1       0.00      0.00      0.00       436
           2       0.16      0.00      0.00      4097
           3       0.25      0.86      0.39      7215
           4       0.18      0.05      0.08      4965
           5       0.17      0.02      0.03      4830
           6       0.11      0.08      0.09      3171

    accuracy                           0.24     28709
   macro avg       0.16      0.14      0.09     28709
weighted avg       0.19      0.24      0.13     28709

Test Classification Report:
              precision    recall  f1-score   support

           0       0.10      0.00      0.00       958
           1       0.00      0.00      0.00       111
           2       0.22      0.00      0.01      1024
           3       0.25      0.86      0

/Users/ayushidubey/Desktop/College/Semester-6/NNDL/labs/project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ayushidubey/Desktop/College/Semester-6/NNDL/labs/project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ayushidubey/Desktop/College/Semester-6/NNDL/labs/project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use 

In [10]:
# Save model
model.save("trained_models/vgg19_fer_15_epochs.keras")

# # Load model later
# from tensorflow.keras.models import load_model
# model = load_model("vgg16_fer.h5")